In [ ]:
pip install transformers datasets torch fastapi uvicorn

In [ ]:
pip install accelerate -U

data preprocessing

In [ ]:
def preprocess_data(input_file, output_file):
    try:
        # Open the input file with a specific encoding
        with open(input_file, 'r', encoding='latin-1') as infile:
            content = infile.read()

        # Process the content as needed
        # For demonstration purposes, let's just write it to the output file
        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.write(content)

        print(f"Data has been processed and written to {output_file}")

    except UnicodeDecodeError as e:
        print(f"Error reading {input_file}: {e}")

# Specify your input and output file names
input_file = "guvi_data.txt"  # Your company-specific data file
output_file = "processed_guvi_data.txt"

# Call the function to preprocess data
preprocess_data(input_file, output_file)

finetuneing the pretrained model

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Create dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )

train_dataset = load_dataset(output_file, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=8,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("/content/drive/My Drive/fine_tuned_model")
tokenizer.save_pretrained("/content/drive/My Drive/fine_tuned_model")

TESTING THE MODEL

In [ ]:
#!pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model and tokenizer
model_name_or_path = "/content/drive/My Drive/fine_tuned_model"
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the text generation function
def generate_text(model, tokenizer, seed_text, max_length=100, temperature=1.0, num_return_sequences=1):
    # Tokenize the input text
    input_ids = tokenizer.encode(seed_text, return_tensors='pt').to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            top_k=50,
            top_p=0.95,
        )

    # Decode the generated text
    generated_texts = []
    for i in range(num_return_sequences):
        generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return generated_texts

# Test the model
seed_text = input()
generated_texts = generate_text(model, tokenizer, seed_text, max_length=50, temperature=1.5, num_return_sequences=3)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}:\n{text}\n")